In [1]:
import sys
sys.path.append('C:/Users/lslat/Code/')
sys.path.append('/mnt/c/Users/lslat/Code/')
sys.path.append('/mnt/c/Users/lslat/QiskitProjects/VariationalWavefunction/')
from quantum_kernel.code.feature_maps import IQP
from quantum_kernel.code.feature_maps import Heisenberg
from quantum_kernel.code.utils import get_quantum_kernel,get_projected_quantum_kernel
from quantum_kernel.code.quantum_kernels import projected_kernels
from quantum_kernel.code.utils import get_dataset
from qiskit.circuit import Parameter,ParameterVector

In [2]:
test=IQP.Sparse_IQP(10,1,"full",1,int_time_scale=0.2)

In [3]:
test_kernel=get_projected_quantum_kernel(test,simulation_method='matrix_product_state')

In [4]:
test_kernel._feature_map.parameters

ParameterView([ParameterVectorElement(x[0]), ParameterVectorElement(x[1]), ParameterVectorElement(x[2]), ParameterVectorElement(x[3]), ParameterVectorElement(x[4]), ParameterVectorElement(x[5]), ParameterVectorElement(x[6]), ParameterVectorElement(x[7]), ParameterVectorElement(x[8]), ParameterVectorElement(x[9])])

In [5]:
x_train, x_test, y_train, y_test = get_dataset('plasticc', 10, 800, 200)


In [6]:
qc_x,qc_y=test_kernel.construct_circuit(x_train[0],x_train[0])
test_circuit=qc_x[0]

ParameterView([ParameterVectorElement(x[0]), ParameterVectorElement(x[1]), ParameterVectorElement(x[2]), ParameterVectorElement(x[3]), ParameterVectorElement(x[4]), ParameterVectorElement(x[5]), ParameterVectorElement(x[6]), ParameterVectorElement(x[7]), ParameterVectorElement(x[8]), ParameterVectorElement(x[9])])
ParameterView([])
ParameterView([])
ParameterView([])


In [38]:
a=test_kernel._feature_map


In [39]:
a.parameters

ParameterView([ParameterVectorElement(x[0]), ParameterVectorElement(x[1]), ParameterVectorElement(x[2]), ParameterVectorElement(x[3]), ParameterVectorElement(x[4]), ParameterVectorElement(x[5]), ParameterVectorElement(x[6]), ParameterVectorElement(x[7]), ParameterVectorElement(x[8]), ParameterVectorElement(x[9])])

In [35]:
feature_map_params_x = ParameterVector("par_x", test.num_parameters)
test_circuit.assign_parameters({feature_map_params_x: x_train[0]})

CircuitError: 'Cannot bind parameters (par_x[0], par_x[1], par_x[2], par_x[3], par_x[4], par_x[5], par_x[6], par_x[7], par_x[8], par_x[9]) not present in the circuit.'

In [37]:
test_circuit._unsorted_parameters()

set()

In [21]:
test_circuit._unroll_param_dict({feature_map_params_x:x_train[0]})

{ParameterVectorElement(par_x[0]): 1.9594150198638007,
 ParameterVectorElement(par_x[1]): -1.696834749979715,
 ParameterVectorElement(par_x[2]): 0.543231810300563,
 ParameterVectorElement(par_x[3]): 1.4921900582289402,
 ParameterVectorElement(par_x[4]): 1.3828200393378418,
 ParameterVectorElement(par_x[5]): -1.2150992767732984,
 ParameterVectorElement(par_x[6]): 0.7768108225598058,
 ParameterVectorElement(par_x[7]): -0.15582995390097662,
 ParameterVectorElement(par_x[8]): -1.0009887970402764,
 ParameterVectorElement(par_x[9]): -0.5026500424513888}

In [14]:
test_kernel.evaluate(x_vec=x_train,y_vec=x_train)

1
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
here
here2


FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: 'bad input to assemble() function; must be either circuits or schedules' 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: 'bad input to assemble() function; must be either circuits or schedules' 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: 'bad input to assemble() function; must be either circuits or schedules' 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: 'bad input to assemble() function; must be either circuits or schedules' 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: 'bad input to assemble() function; must be either circuits or schedules' 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: 'bad input to assemble() function; must be either circuits or schedules' 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: 'ba

here3


QiskitError: 'Max retry limit reached. Failed to submit the qobj correctly'

: 

: 

: 

: 

: 

: 

: 